# **Creacion del Datawarehouse y Llenado de las Tablas de Dimensiones**
El datawarehouse sera implementado en una base de datos PostgreSQL alojada en Aiven. Este datawarehouse tendrá un área de staging, que contendrá una tabla `bautistalavielle_historical_market_depth` y otra `bautistalavielle_aggregates`.
En el [modelo dimensional](https://miro.com/app/board/uXjVKaplXV0=/?share_link_id=415454751190), tendremos:
- Tablas de hechos: `bautistalavielle_historical_market_depth_fact` y `bautistalavielle_aggregates_fact`
- Tablas de dimensiones: `bautistalavielle_cryptos_dim`, `bautistalavielle_dates_dim` y `bautistalavielle_times_dim`

#### **Importacion de Librerias**

In [1]:
#Importo todas las Librerias que utilizaré

from sqlalchemy import text
from utils_db import *
from utils_parquet import *
from utils_processing import *

#### **Coneccion a la Base de Datos PostgreSQL Alojada en Aiven**

In [2]:
#Me conecto a la base de datos con los datos de conexión y credenciales a Postgres almacenados en el archivo 
#de configuración 'pipeline.conf', en la seccion 'postgres', utilizando connect_to_db() de utils_db.py
engine = connect_to_db(
    "pipeline.conf",
    "postgres"
    )

conn = engine.connect()

#### **Creación de Tablas con sus Constraints**

In [3]:
create_query = text(
        """
        BEGIN;

        -- Crear el esquema 'stage'
        CREATE SCHEMA IF NOT EXISTS stage;

        -- Crear la tabla 'bautistalavielle_historical_market_depth' en 'stage'
        CREATE TABLE IF NOT EXISTS stage.bautistalavielle_historical_market_depth (
            historical_market_depth_id BIGINT PRIMARY KEY NOT NULL,
            symbol VARCHAR(50) NOT NULL,
            price FLOAT,
            quantity FLOAT,
            date DATE NOT NULL,
            hour_minute_second TIME NOT NULL
        );

        -- Crear la tabla 'bautistalavielle_aggregates' en 'stage'
        CREATE TABLE IF NOT EXISTS stage.bautistalavielle_aggregates (
            aggregates_id BIGINT PRIMARY KEY NOT NULL,
            symbol VARCHAR(50) NOT NULL,
            open_price FLOAT,
            close_price FLOAT,
            open_close_difference FLOAT,
            high_price FLOAT,
            low_price FLOAT,
            max_daily_difference FLOAT,
            volume FLOAT,
            date DATE NOT NULL
        );

        -- Crear el esquema 'datawarehouse'
        CREATE SCHEMA IF NOT EXISTS datawarehouse;

        -- Crear la tabla 'bautistalavielle_cryptos_dim' en 'datawarehouse'
        CREATE TABLE IF NOT EXISTS datawarehouse.bautistalavielle_cryptos_dim (
            crypto_id SERIAL PRIMARY KEY NOT NULL,
            name VARCHAR(50) DEFAULT NULL,
            symbol VARCHAR(50) NOT NULL
        );

        -- Crear la tabla 'bautistalavielle_dates_dim' en 'datawarehouse'
        CREATE TABLE IF NOT EXISTS datawarehouse.bautistalavielle_dates_dim (
            date_id BIGINT PRIMARY KEY NOT NULL,
            day INT NOT NULL,
            month INT NOT NULL,
            year INT NOT NULL,
            quarter INT NOT NULL,
            day_of_week INT NOT NULL,
            day_of_month INT NOT NULL
        );

        -- Crear la tabla 'bautistalavielle_times_dim' en 'datawarehouse'
        CREATE TABLE IF NOT EXISTS datawarehouse.bautistalavielle_times_dim (
            time_id BIGINT PRIMARY KEY NOT NULL,
            hour INT NOT NULL,
            minute INT NOT NULL,
            second INT NOT NULL
        );

        -- Crear la tabla 'bautistalavielle_historical_market_depth_fact' en 'datawarehouse'
        CREATE TABLE IF NOT EXISTS datawarehouse.bautistalavielle_historical_market_depth_fact (
            historical_market_depth_id BIGINT PRIMARY KEY NOT NULL,
            symbol_id INT NOT NULL,
            price FLOAT,
            quantity FLOAT,
            historical_market_depth_date_id INT NOT NULL,
            historical_market_depth_time_id INT NOT NULL,
            FOREIGN KEY (symbol_id) REFERENCES datawarehouse.bautistalavielle_cryptos_dim(crypto_id),
            FOREIGN KEY (historical_market_depth_date_id) REFERENCES datawarehouse.bautistalavielle_dates_dim(date_id),
            FOREIGN KEY (historical_market_depth_time_id) REFERENCES datawarehouse.bautistalavielle_times_dim(time_id)
        );

        -- Crear la tabla 'bautistalavielle_aggregates_fact' en 'datawarehouse'
        CREATE TABLE IF NOT EXISTS datawarehouse.bautistalavielle_aggregates_fact (
            aggregates_id BIGINT PRIMARY KEY NOT NULL,
            symbol_id INT NOT NULL,
            open_price FLOAT,
            close_price FLOAT,
            open_close_difference FLOAT,
            high_price FLOAT,
            low_price FLOAT,
            max_daily_difference FLOAT,
            volume FLOAT,
            aggregates_date_id INT NOT NULL,
            FOREIGN KEY (symbol_id) REFERENCES datawarehouse.bautistalavielle_cryptos_dim(crypto_id),
            FOREIGN KEY (aggregates_date_id) REFERENCES datawarehouse.bautistalavielle_dates_dim(date_id)
        );

        COMMIT;
        """
    )

with engine.connect() as conn:
    conn.execute(create_query)

#### **Carga de la Tabla `'bautistalavielle_dates_dim'` de `'datawarehouse'`**
Se cargara con todos los dias del año 2020, colocando en `date_id` un entero con el formato **YYMMDD** donde **YY** seran las dos ultimas cifras del año, **MM** el mes, y **DD** el dia correspondiente a la fecha

In [4]:
insert_query = text(
        """
        BEGIN;
        INSERT INTO datawarehouse.bautistalavielle_dates_dim (date_id, day, month, year, quarter, day_of_week, day_of_month)
        SELECT
            TO_NUMBER(TO_CHAR(d, 'YYMMDD'), '999999') AS date_id,
            EXTRACT(DOY FROM d) AS day,
            EXTRACT(MONTH FROM d) AS month,
            EXTRACT(YEAR FROM d) AS year,
            EXTRACT(QUARTER FROM d) AS quarter,
            EXTRACT(ISODOW FROM d) AS day_of_week,
            EXTRACT(DAY FROM d) AS day_of_month
        FROM
            GENERATE_SERIES('2020-01-01'::DATE, '2020-12-31'::DATE, '1 DAY'::INTERVAL) AS d;
        COMMIT;
        """
        )

with engine.connect() as conn:
    conn.execute(insert_query)

#### **Carga de la Tabla `'bautistalavielle_times_dim'` de `'datawarehouse'`**
Se cargara con todos las horas, minutos y segundos de un dia, colocando en `time_id` un entero con el formato **HHMMSS** donde **HH** sera la hora, **MM** los minutos, y **SS** los segundos correspondientes.

In [5]:
insert_query = text(
        """
        BEGIN;
        INSERT INTO datawarehouse.bautistalavielle_times_dim (time_id, hour, minute, second)
        SELECT
            (date_part('hour', payment_time) * 10000) + (date_part('minute', payment_time) * 100) + date_part('second', payment_time) AS time_id,
            date_part('hour', payment_time) AS hour,
            date_part('minute', payment_time) AS minute,
            date_part('second', payment_time) AS second
        FROM (
            SELECT 
                (generate_series || ' seconds')::interval AS payment_time
            FROM generate_series(0, 86399, 1) AS generate_series
        ) AS times;
        COMMIT;
        """
        )

with engine.connect() as conn:
    conn.execute(insert_query)

#### **Carga de la Tabla `'bautistalavielle_cryptos_dim'` de `'datawarehouse'`**

In [6]:
insert_query = text(
        """
        BEGIN;
        INSERT INTO datawarehouse.bautistalavielle_cryptos_dim (name, symbol)
        VALUES
            ('Bitcoin', 'BTCUSD'),
            ('Ethereum', 'ETHUSD'),
            ('Cardano', 'ADAUSD'),
            ('Dogecoin', 'DOGEUSD');
        COMMIT;
        """
        )

with engine.connect() as conn:
    conn.execute(insert_query)